# **Distribución de propiedades por segmento**

In [2]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

## 2022

In [4]:
# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    #if nombre.startswith("df_") and "alfa_q" in nombre and "2022_puebla" in nombre

    if nombre.startswith("df_") and "2022_puebla" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2022_puebla']

In [11]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    segment_df = globals()[nombre_df][['categoria']]
    # Añadir el DataFrame a la listaprecio_m2_construido
    dataframes_list.append(segment_df)

In [12]:
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

,categoria,conteo_total
0,S,779
1,D1,351
2,C3,302
3,B2,279
4,B1,275
5,B3,250
6,D2,232
7,C2,213
8,D3,189
9,A3,161


In [13]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Añadir una nueva fila como un DataFrame
nueva_fila_df = pd.DataFrame({'conteo_total': [0], 'segmento': ['E'], 'clase': ['3']})
# Concatenar el DataFrame original con la nueva fila
conteo_total_dfs = pd.concat([conteo_total_df, nueva_fila_df], ignore_index=True)
conteo_total_dfs

,conteo_total,segmento,clase
0,779,S,1
1,351,D,1
2,302,C,3
3,279,B,2
4,275,B,1
5,250,B,3
6,232,D,2
7,213,C,2
8,189,D,3
9,161,A,3


In [14]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_dfs.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  3609


,segmento,total_por_segmento,porcentaje_del_total
0,A,438,12.136326
1,B,804,22.277639
2,C,659,18.259906
3,D,772,21.390967
4,E,157,4.350236
5,S,779,21.584927


In [15]:
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

fig = go.Figure(data=[go.Pie(labels=segmentos, values=porcentajes_totales, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent+label', textposition='inside')
fig.update_layout(
    yaxis=dict(
            # title='Rango de Edad', 
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        ),
        legend=dict(
            font_size=10,
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
            ),
        
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    # pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2022', carpeta='assets/graficas')

fig.show()

## 2023

In [7]:
# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    #if nombre.startswith("df_") and "alfa_q" in nombre and "2023_puebla" in nombre

    if nombre.startswith("df_") and "2023_puebla" in nombre    
    and ( "jul_2023" in nombre or "sep_2023" in nombre )
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_jul_2023_puebla', 'df_sep_2023_puebla']

In [9]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'segmento':'categoria','Category':'categoria'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [10]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    segment_df = globals()[nombre_df][['categoria']]
    # Añadir el DataFrame a la listaprecio_m2_construido
    dataframes_list.append(segment_df)

In [11]:
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

,categoria,conteo_total
0,S1,314
1,B2,436
2,D3,212
3,C1,223
4,S2,139
5,A1,477
6,A2,197
7,B3,224
8,D2,184
9,B1,223


In [12]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Añadir una nueva fila como un DataFrame
nueva_fila_df = pd.DataFrame({'conteo_total': [0], 'segmento': ['E'], 'clase': ['3']})
# Concatenar el DataFrame original con la nueva fila
conteo_total_dfs = pd.concat([conteo_total_df, nueva_fila_df], ignore_index=True)
conteo_total_dfs

,conteo_total,segmento,clase
0,314,S,1
1,436,B,2
2,212,D,3
3,223,C,1
4,139,S,2
5,477,A,1
6,197,A,2
7,224,B,3
8,184,D,2
9,223,B,1


In [13]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_dfs.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  4406


,segmento,total_por_segmento,porcentaje_del_total
0,A,847,19.223786
1,B,883,20.040853
2,C,696,15.796641
3,D,556,12.619156
4,E,159,3.608715
5,L,93,2.110758
6,S,1172,26.600091


In [14]:
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

fig = go.Figure(data=[go.Pie(labels=segmentos, values=porcentajes_totales, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent+label', textposition='inside')
fig.update_layout(
    yaxis=dict(
            # title='Rango de Edad', 
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        ),
        legend=dict(
            font_size=10,
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
            ),
        
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    # pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2023', carpeta='assets/graficas')

fig.show()

## 2024

In [4]:
# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    #if nombre.startswith("df_") and "alfa_q" in nombre and "2023_puebla" in nombre

    if nombre.startswith("df_") and "2024_puebla" in nombre    
    and ( "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre )
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_feb_2024_puebla', 'df_mar_2024_puebla', 'df_may_2024_puebla']

In [5]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'segmento':'categoria','Category':'categoria'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [6]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    segment_df = globals()[nombre_df][['categoria']]
    # Añadir el DataFrame a la listaprecio_m2_construido
    dataframes_list.append(segment_df)

In [7]:
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

,categoria,conteo_total
0,S1,1568
1,E2,1335
2,E3,1009
3,D2,721
4,B2,645
5,S2,608
6,D3,616
7,C2,593
8,B1,534
9,C1,590


In [8]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Añadir una nueva fila como un DataFrame
nueva_fila_df = pd.DataFrame({'conteo_total': [0], 'segmento': ['E'], 'clase': ['3']})
# Concatenar el DataFrame original con la nueva fila
conteo_total_dfs = pd.concat([conteo_total_df, nueva_fila_df], ignore_index=True)
conteo_total_dfs

,conteo_total,segmento,clase
0,1568,S,1
1,1335,E,2
2,1009,E,3
3,721,D,2
4,645,B,2
5,608,S,2
6,616,D,3
7,593,C,2
8,534,B,1
9,590,C,1


In [9]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_dfs.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  12836


,segmento,total_por_segmento,porcentaje_del_total
0,A,1506,11.732627
1,B,1672,13.025865
2,C,1746,13.602368
3,D,1830,14.256778
4,E,2850,22.203179
5,L,557,4.339358
6,S,2675,20.839825


In [10]:
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

fig = go.Figure(data=[go.Pie(labels=segmentos, values=porcentajes_totales, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent+label', textposition='inside')
fig.update_layout(
    yaxis=dict(
            # title='Rango de Edad', 
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        ),
        legend=dict(
            font_size=10,
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
            ),
        
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    # pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2024', carpeta='assets/graficas')

fig.show()